In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('video_data.csv', error_bad_lines=False,parse_dates=['DateTime'])


b'Skipping line 464716: expected 3 fields, saw 4\nSkipping line 464947: expected 3 fields, saw 4\nSkipping line 465187: expected 3 fields, saw 4\nSkipping line 465442: expected 3 fields, saw 4\nSkipping line 465648: expected 3 fields, saw 4\n'


In [2]:
# step 1: Verify data load
# step 2: Display data types
# step 3: Display summary statistics
# step 4: Data Cleaning 
# step 5: Data tranformation


In [3]:
# step 1: Verifying data load

def check_load(df,num_records,num_columns):
    message = 'Validation Failed'
    if num_records == df.shape[0]:
        if num_columns == df.shape[1]:
            message = 'Validation Passed'
    return message

#Print the data validation message
print(check_load(df,1333653,3))

Validation Passed


In [4]:
# Step 2: Checking data shape 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333653 entries, 0 to 1333652
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype              
---  ------      --------------    -----              
 0   DateTime    1333653 non-null  datetime64[ns, UTC]
 1   VideoTitle  1333653 non-null  object             
 2   events      1333653 non-null  object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 30.5+ MB


In [46]:
df.head(5)

,DateTime,VideoTitle,events
0,2017-01-11 00:00:31+00:00,App Web|Clips|a-current-affair;2016|William Ty...,"157,120,160,104,162,161,163,164,165,166,171,229"
1,2017-01-11 00:00:53+00:00,news| Shark attacks spearfisherman,"127,157,120,160,104,162,161,171,206"
2,2017-01-11 00:00:21+00:00,news| Shark attacks spearfisherman,"127,157,120,160,104,162,161,163,164,165,166,17..."
3,2017-01-11 00:01:27+00:00,news| Chilean navy films UFO,"157,120,160,104,162,161,170,171,237"
4,2017-01-11 00:00:33+00:00,news| Video shows alleged axe attack at Sydney...,"157,120,160,104,162,161,163,164,165,166,171,229"


In [5]:
# Step 3: Display summary statistics
df.describe()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,DateTime,VideoTitle,events
count,1333653,1333653,1333653
unique,84986,7119,109
top,2017-01-11 11:12:39+00:00,news| Piers Morgan reduces mother to tears,"157,120,160,104,162,161,171,206"
freq,61,91799,390414
first,2017-01-11 00:00:00+00:00,NaN,NaN
last,2017-01-11 23:59:59+00:00,NaN,NaN


In [6]:
# Step 4: Data Cleaning 
# Check missing data
print('Display the Number of missing data: \n {} '.format(df.isnull().sum()))

# Check duplicated data
print('\n')
print('Display the Number of duplicated data ---> {}'.format(df.duplicated().sum()))


Display the Number of missing data: 
 DateTime      0
VideoTitle    0
events        0
dtype: int64 


Display the Number of duplicated data ---> 156470


In [7]:
# Address the duplicated data
df = df.drop_duplicates()

#Check again
print('Display the Number of duplicated data---> {}'.format(df.duplicated().sum()))


Display the Number of duplicated data---> 0


In [8]:
# Step 5: Data tranformation
#1. Determined from the “events” column containing “206”
#2. All rows without 206 should be discarded

Events_With_206 = df.events.str.contains('206')

df_without_206 = df[~Events_With_206].reset_index(drop=True)
df_with_206 = df[Events_With_206].reset_index(drop=True)
print("The {} records has been deleted. \n\nThe new dataset has {} rows and {} columns.".format(df_without_206.shape[0],df_with_206.shape[0],df_with_206.shape[1]))

#3. Split VideoTitle by pipe ‘|’
#If VideoTitle.split(‘|’)0 contains something that looks like a platform (iPhone, Android Phone etc) then use that as the platform
#If VideoTitle.split(‘|’)0 doesn’t contain a platform but looks like a site, assume the platform is Desktop


df_with_206['DimPlatform']=''
df_with_206['DimPlatform'][df_with_206['VideoTitle'].str.contains('iPhone')]='iPhone'
df_with_206['DimPlatform'][df_with_206['VideoTitle'].str.contains('Android')]='Android'
df_with_206['DimPlatform'][df_with_206['VideoTitle'].str.contains('iPad')]='Ipad'
df_with_206['DimPlatform'][~df_with_206['VideoTitle'].str.contains('iPhone|Android|iPad')]='Desktop'


#Split VideoTitle ----> DimSite & DimVideo

df_with_206.VideoTitle = df_with_206.VideoTitle.str.split('|')
title_list = []
site_name = []
for row in df_with_206['VideoTitle']:
    title_list.append(row[-1])
    site_name.append(row[0])
    
    
df_with_206['DimSite'] = pd.DataFrame(site_name)    
df_with_206['DimVideo'] = pd.DataFrame(title_list)




The 771644 records has been deleted. 

The new dataset has 405539 rows and 3 columns.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A valu

In [ ]:
# # Dimension_Tables:
# 1. df_DimSite
# 2. df_DimVideo
# 3. df_DimPlatform
# 4. df_Dimdate

# # Fact_table

In [43]:
# Dimension_Tables-- > df_DimSite 


df_DimSite = pd.DataFrame(df_with_206['DimSite'].unique())
df_DimSite.rename(columns={0:'DimSite'},inplace=True)
df_DimSite['DimSite_id'] = pd.DataFrame(np.arange(1,df_DimSite['DimSite'].nunique()+1))

# save as df_DimSite.csv
df_DimSite.to_csv('df_DimSite.csv',index=0)

# Dimension_Tables-- > df_DimVideo 


df_DimVideo = pd.DataFrame(df_with_206['DimVideo'].unique())
df_DimVideo.rename(columns={0:'DimVideo'},inplace=True)
df_DimVideo['DimVideo_id'] = pd.DataFrame(np.arange(1,df_DimVideo['DimVideo'].nunique()+1))

# save as df_DimSite.csv
df_DimVideo.to_csv('df_DimVideo.csv',index=0)


# # Dimension_Tables-- > df_DimPlatform 

df_DimPlatform = pd.DataFrame(df_with_206['DimPlatform'].unique())
df_DimPlatform.rename(columns={0:'DimPlatform'},inplace=True)
df_DimPlatform['df_DimPlatform_id'] = pd.DataFrame(np.arange(1,df_DimPlatform['DimPlatform'].nunique()+1))

# save as df_DimPlatform.csv
df_DimPlatform.to_csv('df_DimPlatform.csv',index=0)



# Dimension_Tables-- > df_Dimdate 

df_Dimdate = df_with_206[['DateTime']]
df_Dimdate = pd.DataFrame(df_Dimdate['DateTime'].unique(),columns=['DateTime'])


df_Dimdate['Date']=df_Dimdate['DateTime'].dt.date
df_Dimdate['Time']=df_Dimdate['DateTime'].dt.time
df_Dimdate['Year']=df_Dimdate['DateTime'].dt.year
df_Dimdate['Quarter']=df_Dimdate['DateTime'].dt.quarter
df_Dimdate['Month']=df_Dimdate['DateTime'].dt.month
df_Dimdate['Day']=df_Dimdate['DateTime'].dt.day
df_Dimdate['Hour']=df_Dimdate['DateTime'].dt.hour
df_Dimdate['Minute']=df_Dimdate['DateTime'].dt.minute

# save as df_Dimdate.csv
df_Dimdate.to_csv('df_Dimdate.csv',index=0)



# Fact_table

df1_date = pd.merge(df_with_206,df_Dimdate,on='DateTime',how='outer')
df1_date_video = pd.merge(df_with_206,df_DimVideo,on='DimVideo',how='outer')
df1_date_video_site = pd.merge(df1_video,df_DimSite,on='DimSite',how='outer')
df1_date_video_site_platform = pd.merge(df1_video_site,df_DimPlatform,on='DimPlatform',how='outer')

fact_table=df1_video_site_platform.loc[:,['DateTime','df_DimPlatform_id','DimSite_id','DimVideo_id','events']]

# save as fact_table.csv
fact_table.to_csv('fact_table.csv',index=0)


